In [ ]:
import sympy as sym
import numpy as np
from sympy import symbols, Eq
import matplotlib.pyplot as plt

# Sims 1999

$$\Gamma_0 y_t=\Gamma_1 y_{t-1}+C+\Psi z_t+\Pi\eta_t$$
For the moment, assume $\Gamma_0$ is invertible

In [ ]:
class Gensys:
    def __init__(self,parameters:str, variables:str, shocks:str,equations):
        ''' 
        variables: str
        string of variables separated by commas.
        '''
        self.vars = {v:symbols(v) for v in variables.replace(" ", "").split(',')}
        self.n_var = len(self.vars)
        self.shocks = {s:symbols(s) for s in shocks.replace(" ", "").split(',')}
        self.n_shocks = len(self.shocks)
        self.parameters = {p:symbols(p) for p in parameters.replace(" ", "").split(',')}
        self.symbolic_system = None
        self.numeric_system = None
        self.matrix_solution = None
        self.calibration = None
        #self.equations = {Eq(*[eval(i) for i in eq.split('=')]) for eq in equations}
        pass

    def get_matrices(self,equations, var, var_, z, eta):
        Gamma0,_ = sym.linear_eq_to_matrix(equations, var)
        Gamma1,_ = sym.linear_eq_to_matrix(_,var_)
        Psi, _ = sym.linear_eq_to_matrix(_, z)
        Eta, C = sym.linear_eq_to_matrix(_, eta)
        Psi = -Psi
        C = -C
        self.symbolic_system={'Gamma0':Gamma0, 'Gamma1':Gamma1,'Psi':Psi, 'Eta':Eta, 'C':C}
        pass
        
    def calibrate(self,calibration):
        self.calibration = calibration
        Gamma0 = self.symbolic_system['Gamma0'].subs(calibration)
        Gamma1 = self.symbolic_system['Gamma1'].subs(calibration)
        Psi = self.symbolic_system['Psi'].subs(calibration)
        Eta = self.symbolic_system['Eta']
        C = self.symbolic_system['C'].subs(calibration)
        self.numeric_system = {'Gamma0':Gamma0, 'Gamma1':Gamma1,'Psi':Psi, 'Eta':Eta, 'C':C}
        pass

    def solve(self):
        return self._solve(**self.numeric_system)
     
    def _solve(self, Gamma0, Gamma1, Psi, Eta, C):
        Gamma0_inv=Gamma0.inv()
        Gamma1, C, Psi, Eta = Gamma0_inv@Gamma1, Gamma0_inv@C, Gamma0_inv@Psi, Gamma0_inv@Eta 
        P, Lambda = Gamma1.diagonalize()
        ind = list(np.argsort(np.abs(np.diag(np.abs(Lambda)))))
        Lambda = sym.diag(*[Lambda[i,i] for i in ind])
        n_s = len([Lambda[i,i] for i in range(Lambda.rows) if np.abs(Lambda[i,i])<1])
        n_u = Lambda.rows-n_s        
        P=P.permute_cols(ind)  
        P = np.array(P).astype(np.complex128) 
        P_inv = np.linalg.inv(P)
        Lambda_S=Lambda[:n_s,:n_s]
        Lambda_U=Lambda[n_s:,n_s:]
        Theta_1 = P[:,:n_s]@Lambda_S@P_inv[:n_s,:]
        Theta_c = (P[:,:n_s]@P_inv[:n_s,:]+P[:,n_s:]@(sym.eye(n_u)-Lambda_U)@P_inv[n_s:,:])@C
        Phi = P_inv[:n_s,:]@Eta@(P_inv[n_s:,:]@Eta).inv()
        Theta_z = (P[:,:n_s]@P_inv[:n_s,:]-P[:,:n_s]@Phi@P_inv[n_s:,:])@Psi
        Theta_1 = np.real(np.array(Theta_1).astype(np.complex64))
        Theta_c = np.real(np.array(Theta_c).astype(np.complex64))
        Theta_z = np.real(np.array(Theta_z).astype(np.complex64)) 
        self.matrix_solution={'Theta_1':Theta_1,'Theta_c':Theta_c, 'Theta_z':Theta_z}
    
    def impulse_response(self, z:np.array, T:int = 25):
        '''
        z: initial shock
        '''
        n_var = self.n_var
        Theta_1 = self.matrix_solution['Theta_1']
        Theta_c = self.matrix_solution['Theta_c']
        Theta_z = self.matrix_solution['Theta_z']
        z_t = np.zeros((T,self.n_shocks))
        z_t[:,0] = z
        y_t = np.zeros((T,n_var))
        t=np.arange(0,T)
        for i in range(1,T):
            y_t[i]=(Theta_1@y_t[[i-1]].T+Theta_c+Theta_z@z_t[[i]].T).flatten()

        max_cols=3
        fig,ax=plt.subplots(nrows=(n_var-1)//max_cols+1,ncols=min(n_var,max_cols), figsize=(10,10))
        for i in range(n_var):
            ax[i//max_cols,i%max_cols].plot(t,y_t[:,i])
        fig.show()

## Galí (2015) sticky prices

**Define simbolic variables, parameters and shocks**

In [ ]:
E_y, E_pi, y_, pi_ = symbols('E_{t-1}y_t,E_{t-1}\pi^p_t, y_{t-1}, \pi_{t-1}')
Ey1, Epi1, y, pi = symbols('E_ty_{t+1},E_t\pi^p_{t+1}, y_t, \pi_t')
phi_y, phi_p =symbols('\phi_y, \phi_p')
lam = symbols('\lambda')
sigma, kappa, beta, rho = symbols('sigma, kappa, beta, rho')
eta_y, eta_pi = symbols('\eta_y, \eta_{\pi}')
v, v_ = symbols('v_t, v_{t-1}')
rho_v = symbols('\\rho_v')
eps_v = symbols('\epsilon_v')

**Define static equations**

In [ ]:
i = rho+phi_p*pi+phi_y*y+v

**Define system equations**

In [ ]:
is_curve = Eq(y, Ey1-1/sigma*(i-Epi1-rho))
nkpc = Eq(pi, beta*Epi1+kappa*y)
exp_y = Eq(y, E_y+eta_y)
exp_pi = Eq(pi, E_pi+eta_pi)
v_ar=Eq(v, rho_v*v_+eps_v)

**Define Variables, lags, shocks and expectation errors**

In [ ]:
var  = Ey1, Epi1, y, pi, v
var_ = E_y, E_pi, y_, pi_, v_
z = eps_v,
eta = eta_y, eta_pi

**Calibration**

In [ ]:
calibration={'beta':0.99, 'phi':5, 'alpha':1/4, 'eps':9,
             '\phi_p':1.5, '\phi_y':0.5/4,'\\rho_v':0.5,
             'sigma':1,
              'theta':3/4}
calibration['Theta']=(1-calibration['alpha'])/(1-calibration['alpha']+calibration['alpha']*calibration['eps'])
calibration['lambda']=(1-calibration['theta'])*(1-calibration['beta']*calibration['theta'])/calibration['theta']*calibration['Theta']
calibration['kappa']=calibration['lambda']*(calibration['sigma']+(calibration['phi']+calibration['alpha'])/(1-calibration['alpha']))


In [ ]:
g=Gensys('f,s,f','y,e,d,f,g','e',0)
g.get_matrices([is_curve,nkpc, exp_y, exp_pi, v_ar], var, var_, z, eta)
g.calibrate(calibration)
g.solve()

In [ ]:
g.impulse_response(z=-0.25)

## Galí Sticky Wage

In [ ]:
E_y, E_pi_p, E_pi_w, y_, pi_p_, pi_w_, w_= symbols('E_{t-1}y_t E_{t-1}\pi^p_t E_{t-1}\pi^w_t y_{t-1} \pi^p_{t-1} pi^w_{t-1} omega_{t-1}')
Ey1, Epi_p1, Epi_w1, y, pi_p, pi_w, w = symbols('E_ty_{t+1} E_t\pi^p_{t+1} E_tpi^w_{t+1} y_t pi^p_t pi^w_t omega_t')
phi_y, phi_p, phi_w =symbols('phi_y phi_p phi_w')
lam_p, lam_w = symbols('lambda_p lambda_w')
sigma, kappa, beta, rho = symbols('sigma, kappa, beta, rho')
eta_y, eta_pi_p, eta_pi_w = symbols('\eta_y, \eta_{\pi^p}, \eta_{\pi^w}')
chi_p, chi_w = symbols('chi_p chi_w')
v, v_ = symbols('v_t, v_{t-1}')
rho_v = symbols('\\rho_v')
eps_v = symbols('\epsilon_v')

In [ ]:
i=rho+phi_p*pi_p+phi_w*pi_w+phi_y*y+v

equations=[
    Eq(y, Ey1-1/sigma*(i-Epi_p1-rho)), #IS curve
    Eq(pi_p, beta*Epi_p1+chi_p*y+lam_p*w),  #Price NKPC
    Eq(pi_w, beta*Epi_w1+chi_w*y-lam_w*w),  #Price NKPC
    Eq(w, w_+pi_w-pi_p), #Real wage gap
    Eq(y, E_y+eta_y), # Expectation error for y
    Eq(pi_p, E_pi_p+eta_pi_p), #Expectation error for pi^p
    Eq(pi_w, E_pi_w+eta_pi_w), #Expectation error for pi^w
    Eq(v, rho_v*v_+eps_v) #V
]

In [ ]:
var  = Ey1, Epi_p1, Epi_w1, y, pi_p, pi_w, w, v
var_ = E_y, E_pi_p, E_pi_w, y_, pi_p_, pi_w_, w_, v_
z = eps_v,
eta = eta_y, eta_pi

In [ ]:
calibration={'beta':0.99, 'phi':5, 'alpha':1/4, 'sigma':1,
             'eps_p':9, 'eps_w':4.5,
             'phi_p':1.5, 'phi_y':0.5/4, 'phi_w':0, '\\rho_v':0.5,
             'theta_p':3/4, 'theta_w':3/4}

calibration['Theta']=(1-calibration['alpha'])/(1-calibration['alpha']+calibration['alpha']*calibration['eps_p'])
calibration['lambda_p']=(1-calibration['theta_p'])*(1-calibration['beta']*calibration['theta_p'])/calibration['theta_p']*calibration['Theta']
calibration['lambda_w']= (1-calibration['theta_w'])*(1-calibration['beta']*calibration['theta_w'])/(calibration['theta_w']*(1+calibration['eps_w']*calibration['phi']))

calibration['chi_p']=calibration['alpha']*calibration['lambda_p']*(1-calibration['alpha'])
calibration['chi_w']=calibration['lambda_w']*(calibration['sigma']+calibration['phi']/(1-calibration['alpha']))


In [ ]:
g=Gensys('f,s,f','q,w,e,r,t,y,u,i','e',0)
g.get_matrices(equations, var, var_, z, eta)
g.calibrate(calibration)
g.solve()

In [ ]:
g.impulse_response(z=-0.25)